# Part 5: Takes the MCMC outputs and generates the figures and results published in [Montpetit et al. (Preprint)](TBD)

Benoit Montpetit, CPS/CRD/ECCC, 2025  
Julien Meloche, CPS/CRD/ECCC, 2025  
Mike Brady, CPS/CRD/ECCC, 2025  
Nicolas Leroux, RPN-E/MRD/ECCC, 2025  

This notebook takes the posterior distributions obtained in Part 4, and generates the tables, results and figures published in [Montpetit et al. (Preprint)](TBD)

In [ ]:
import arviz as az
import pandas as pd
import numpy as np
import os
import xarray as xr
import datetime
from matplotlib import pyplot as plt
import matplotlib
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import truncnorm, norm
import glob

In [ ]:
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

In [ ]:
font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)
plt.rcParams["axes.labelsize"] = 22
plt.rcParams["axes.labelweight"] = 'bold'
plt.rcParams['xtick.labelsize']=16
plt.rcParams['ytick.labelsize']=16

Read the statistically representative snowpit profiles surveyed during the TVC Experiment 2018/19 [(Montpetit et al., 2024)](https://tc.copernicus.org/articles/18/3857/2024/)

In [ ]:
df_pits = pd.read_json('Data/df_stat_pits.json')

Read the surveyed snowpit statistics for each grain type

In [ ]:
snowpit_stats = xr.open_dataset('Data/Snowpit_statistics.nc')

# The next cells read the MCMC NetCDF files for given tests and concatenates all the individual sites into one dataframe

## Test using all 120 ensemble members of the default version of SVS-2

In [ ]:
if os.path.isfile('Data/MCMC_Outputs/SWE_default.csv'):

    df_swe_default=pd.read_csv('Data/MCMC_Outputs/SWE_default.csv')

else:

    files = [f for f in os.listdir('Data') if f.endswith('Default.nc')]
    
    df_swe_default = pd.DataFrame({'Site':[]})
    
    for file in files:
    
        site=file.split('_')[2]
    
        if site not in df_swe_default.Site.values:
        
            idata=az.from_netcdf('Data/' + file)
            
            idata.posterior = idata.posterior.assign(SWE=(idata.posterior['Density_R']*idata.posterior['Thickness_R']+
                                                          idata.posterior['Density_H']*idata.posterior['Thickness_H']))
            df_swe_default = pd.concat([df_swe_default,
                                        pd.DataFrame({'Site':[site],
                                                      'Measure_median':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['median'].values],
                                                      'Measure_std':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['std'].values],
                                                      'Measure_quart1':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart1'].values],
                                                      'Measure_quart3':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart3'].values],
                                                      'Retrieved_median':[idata.posterior['SWE'].median().values],
                                                      'Retrieved_std':[idata.posterior['SWE'].std().values],
                                                      'Retrieved_quart1':[idata.posterior['SWE'].quantile(0.25).values],
                                                      'Retrieved_quart3':[idata.posterior['SWE'].quantile(0.75).values],
                                                      'rhat':[az.rhat(idata)['SWE'].values],
                                                      'ess':[az.ess(idata)['SWE'].values]})],
                                       ignore_index=True)

    df_swe_default['bias'] = df_swe_default.Measure_median-df_swe_default.Retrieved_median
    df_swe_default.to_csv('Data/MCMC_Outputs/SWE_default.csv')
df_swe_default.sort_values('Site')

## Test using the top 30 ensemble members of the default version of SVS-2

In [ ]:
if os.path.isfile('Data/MCMC_Outputs/SWE_defaultTop30.csv'):

    df_swe_defaulttop30=pd.read_csv('Data/MCMC_Outputs/SWE_defaultTop30.csv')

else:

    files = [f for f in os.listdir('Data') if f.endswith('DefaultTop30.nc')]
    
    df_swe_defaulttop30 = pd.DataFrame({'Site':[]})
    
    for file in files:
    
        site=file.split('_')[2]
    
        if site not in df_swe_defaulttop30.Site.values:
        
            idata=az.from_netcdf('Data/' + file)
            
            idata.posterior = idata.posterior.assign(SWE=(idata.posterior['Density_R']*idata.posterior['Thickness_R']+
                                                          idata.posterior['Density_H']*idata.posterior['Thickness_H']))
            df_swe_defaulttop30 = pd.concat([df_swe_defaulttop30,
                                        pd.DataFrame({'Site':[site],
                                                      'Measure_median':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['median'].values],
                                                      'Measure_std':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['std'].values],
                                                      'Measure_quart1':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart1'].values],
                                                      'Measure_quart3':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart3'].values],
                                                      'Retrieved_median':[idata.posterior['SWE'].median().values],
                                                      'Retrieved_std':[idata.posterior['SWE'].std().values],
                                                      'Retrieved_quart1':[idata.posterior['SWE'].quantile(0.25).values],
                                                      'Retrieved_quart3':[idata.posterior['SWE'].quantile(0.75).values],
                                                      'rhat':[az.rhat(idata)['SWE'].values],
                                                      'ess':[az.ess(idata)['SWE'].values]})],
                                       ignore_index=True)

    df_swe_defaulttop30['bias'] = df_swe_defaulttop30.Measure_median-df_swe_defaulttop30.Retrieved_median
    df_swe_defaulttop30.to_csv('Data/MCMC_Outputs/SWE_defaultTop30.csv')
df_swe_defaulttop30.sort_values('Site')

## Test using all 120 ensemble members of the Arctic version of SVS-2

In [ ]:
if os.path.isfile('Data/MCMC_Outputs/SWE_arctic.csv'):

    df_swe_arctic=pd.read_csv('Data/MCMC_Outputs/SWE_arctic.csv')

else:
    
    files = [f for f in os.listdir('Data') if f.endswith('1obs_Arctic.nc')]
    
    df_swe_arctic = pd.DataFrame({'Site':[]})
    
    for file in files:
    
        site=file.split('_')[2]
    
        if site not in df_swe_arctic.Site.values:
        
            idata=az.from_netcdf('Data/' + file)
            
            idata.posterior = idata.posterior.assign(SWE=(idata.posterior['Density_R']*idata.posterior['Thickness_R']+
                                                          idata.posterior['Density_H']*idata.posterior['Thickness_H']))
            df_swe_arctic = pd.concat([df_swe_arctic,
                                        pd.DataFrame({'Site':[site],
                                                      'Measure_median':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['median'].values],
                                                      'Measure_std':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['std'].values],
                                                      'Measure_quart1':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart1'].values],
                                                      'Measure_quart3':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart3'].values],
                                                      'Retrieved_median':[idata.posterior['SWE'].median().values],
                                                      'Retrieved_std':[idata.posterior['SWE'].std().values],
                                                      'Retrieved_quart1':[idata.posterior['SWE'].quantile(0.25).values],
                                                      'Retrieved_quart3':[idata.posterior['SWE'].quantile(0.75).values],
                                                      'rhat':[az.rhat(idata)['SWE'].values],
                                                      'ess':[az.ess(idata)['SWE'].values]})],
                                       ignore_index=True)
    
    df_swe_arctic['bias'] = df_swe_arctic.Measure_median-df_swe_arctic.Retrieved_median
    df_swe_arctic.to_csv('Data/MCMC_Outputs/SWE_arctic.csv')
df_swe_arctic.sort_values('Site')

## Test using the top 30 ensemble members of the default version of SVS-2

In [ ]:
if os.path.isfile('Data/MCMC_Outputs/SWE_ArcticTop30.csv'):

    df_swe_arctictop30 = pd.read_csv('Data/MCMC_Outputs/SWE_ArcticTop30.csv')

else:
    
    files = [f for f in os.listdir('Data') if f.endswith('ArcticTop30.nc')]
    
    df_swe_arctictop30 = pd.DataFrame({'Site':[]})
    
    for file in files:
    
        site=file.split('_')[2]
    
        if site not in df_swe_arctictop30.Site.values:
        
            idata=az.from_netcdf('Data/' + file)
            idata = idata.sel(draw=slice(4999))
            
            idata.posterior = idata.posterior.assign(SWE=(idata.posterior['Density_R']*idata.posterior['Thickness_R']+
                                                          idata.posterior['Density_H']*idata.posterior['Thickness_H']))
            df_swe_arctictop30 = pd.concat([df_swe_arctictop30,
                                        pd.DataFrame({'Site':[site],
                                                      'Measure_median':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['median'].values],
                                                      'Measure_std':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['std'].values],
                                                      'Measure_quart1':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart1'].values],
                                                      'Measure_quart3':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart3'].values],
                                                      'Retrieved_median':[idata.posterior['SWE'].median().values],
                                                      'Retrieved_std':[idata.posterior['SWE'].std().values],
                                                      'Retrieved_quart1':[idata.posterior['SWE'].quantile(0.25).values],
                                                      'Retrieved_quart3':[idata.posterior['SWE'].quantile(0.75).values],
                                                      'rhat':[az.rhat(idata)['SWE'].values],
                                                      'ess':[az.ess(idata)['SWE'].values]})],
                                       ignore_index=True)
    
    df_swe_arctictop30['bias'] = df_swe_arctictop30.Measure_median-df_swe_arctictop30.Retrieved_median
    df_swe_arctictop30.to_csv('Data/MCMC_Outputs/SWE_ArcticTop30.csv')
df_swe_arctictop30.sort_values('Site')

## Test using all 120 ensemble members of the default Arctic version of SVS-2 with inter-layer constraints

In [ ]:
if os.path.isfile('Data/MCMC_Outputs/SWE_Arctic_Constraints.csv'):

    df_swe_arcticcond=pd.read_csv('Data/MCMC_Outputs/SWE_Arctic_Constraints.csv')

else:

    files = [f for f in os.listdir('Data') if f.endswith('Arctic_WSRMSE_conditions.nc')]
    
    df_swe_arcticcond = pd.DataFrame({'Site':[]})
    
    for file in files:
    
        site=file.split('_')[2]
    
        if site not in df_swe_arcticcond.Site.values:
        
            idata=az.from_netcdf('Data/' + file)
            idata = idata.sel(draw=slice(5000))
            
            idata.posterior = idata.posterior.assign(SWE=(idata.posterior['Density_R']*idata.posterior['Thickness_R']+
                                                          idata.posterior['Density_H']*idata.posterior['Thickness_H']))
            df_swe_arcticcond = pd.concat([df_swe_arcticcond,
                                        pd.DataFrame({'Site':[site],
                                                      'Measure_median':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['median'].values],
                                                      'Measure_std':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['std'].values],
                                                      'Measure_quart1':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart1'].values],
                                                      'Measure_quart3':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart3'].values],
                                                      'Retrieved_median':[idata.posterior['SWE'].median().values],
                                                      'Retrieved_std':[idata.posterior['SWE'].std().values],
                                                      'Retrieved_quart1':[idata.posterior['SWE'].quantile(0.25).values],
                                                      'Retrieved_quart3':[idata.posterior['SWE'].quantile(0.75).values],
                                                      'rhat':[az.rhat(idata)['SWE'].values],
                                                      'ess':[az.ess(idata)['SWE'].values]})],
                                       ignore_index=True)
    
    df_swe_arcticcond['bias'] = df_swe_arcticcond.Measure_median-df_swe_arcticcond.Retrieved_median
    df_swe_arcticcond.to_csv('Data/MCMC_Outputs/SWE_Arctic_Constraints.csv')
df_swe_arcticcond.sort_values('Site')

## Test using all 120 ensemble members of the Arctic version of SVS-2 with inter-layer constraints and 4 observations

In [ ]:
if os.path.isfile('Data/MCMC_Outputs/SWE_Arctic_conditions_4obs.csv'):

    df_swe_arctic6=pd.read_csv('Data/MCMC_Outputs/SWE_Arctic_conditions_4obs.csv')

else:

    files = [f for f in os.listdir('Data/MCMC_Outputs') if f.endswith('4obs_Arctic_WSRMSE_conditions.nc')]
    
    df_swe_arctic6 = pd.DataFrame({'Site':[]})
    
    for file in files:
    
        site=file.split('_')[2]
    
        if site not in df_swe_arctic6.Site.values:
        
            idata=az.from_netcdf('Data/MCMC_Outputs/' + file)
            idata = idata.sel(draw=slice(5000))
            
            idata.posterior = idata.posterior.assign(SWE=(idata.posterior['Density_R']*idata.posterior['Thickness_R']+
                                                          idata.posterior['Density_H']*idata.posterior['Thickness_H']))
            df_swe_arctic6 = pd.concat([df_swe_arctic6,
                                        pd.DataFrame({'Site':[site],
                                                      'Measure_median':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['median'].values],
                                                      'Measure_std':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['std'].values],
                                                      'Measure_quart1':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart1'].values],
                                                      'Measure_quart3':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart3'].values],
                                                      'Retrieved_median':[idata.posterior['SWE'].median().values],
                                                      'Retrieved_std':[idata.posterior['SWE'].std().values],
                                                      'Retrieved_quart1':[idata.posterior['SWE'].quantile(0.25).values],
                                                      'Retrieved_quart3':[idata.posterior['SWE'].quantile(0.75).values],
                                                      'rhat':[az.rhat(idata)['SWE'].values],
                                                      'ess':[az.ess(idata)['SWE'].values]})],
                                       ignore_index=True)

df_swe_arctic6['bias'] = df_swe_arctic6.Measure_median-df_swe_arctic6.Retrieved_median
df_swe_arctic6.to_csv('Data/MCMC_Outputs/SWE_Arctic_conditions_4obs.csv')
df_swe_arctic6.sort_values('Site')

### Reading the UMASS measured backscatter during the TVC Experiment 2018/19

In [ ]:
umass = pd.read_pickle('Data/UMass_TVC18-19_DB.pkl')
umass_tvc02=umass.drop(columns='geometry')[(umass.radar_ts<=datetime.datetime(2019,2,1)) & (umass.radar_ts>=datetime.datetime(2019,1,1))]
max_date = umass_tvc02.radar_ts.max()
min_date = umass_tvc02.radar_ts.min()

### Reading all ensemble members of both SVS-2 version and calculating the mean, median, 1st and 3rd quartiles

In [ ]:
svs_arctic = xr.open_dataset('Data/SVS-2_ArcticEnsembles_TVC02.nc')
svs_default = xr.open_dataset('Data/SVS-2_DefaultEnsembles_TVC02.nc')

svs_mean_swe_arctic = svs_arctic.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().values.mean()
svs_median_swe_arctic = np.median(svs_arctic.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().values)
svs_quart1_swe_arctic = svs_arctic.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().quantile(0.25).values
svs_quart3_swe_arctic = svs_arctic.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().quantile(0.75).values

svs_mean_swe_default = svs_default.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().values.mean()
svs_median_swe_default = np.median(svs_default.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().values)
svs_quart1_swe_default = svs_default.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().quantile(0.25).values
svs_quart3_swe_default = svs_default.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().quantile(0.75).values

### Reading the top 30 ensemble members of both SVS-2 version and calculating the mean, median, 1st and 3rd quartiles

In [ ]:
svs_arctictop30 = xr.open_dataset('Data/SVS-2_ArcticTop30Ensembles_TVC02.nc')
svs_defaulttop30 = xr.open_dataset('Data/SVS-2_DefaultTop30Ensembles_TVC02.nc')

svs_mean_swe_arctictop30 = svs_arctic.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().values.mean()
svs_median_swe_arctictop30 = np.median(svs_arctic.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().values)
svs_quart1_swe_arctictop30 = svs_arctic.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().quantile(0.25).values
svs_quart3_swe_arctictop30 = svs_arctic.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().quantile(0.75).values

svs_mean_swe_defaulttop30 = svs_default.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().values.mean()
svs_median_swe_defaulttop30 = np.median(svs_default.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().values)
svs_quart1_swe_defaulttop30 = svs_default.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().quantile(0.25).values
svs_quart3_swe_defaulttop30 = svs_default.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)))['SNOMA'].to_dataframe().dropna().quantile(0.75).values

### Organizing the statistics into dataframes and a large dictionary

In [ ]:
svs_dfs = {}

svs_dfs['Default']=pd.DataFrame({'mean':[svs_mean_swe_default], 'median':[svs_median_swe_default], 'quart1':[svs_quart1_swe_default], 'quart3':[svs_quart3_swe_default]})
svs_dfs['Default Top30']=pd.DataFrame({'mean':[svs_mean_swe_defaulttop30], 'median':[svs_median_swe_defaulttop30], 'quart1':[svs_quart1_swe_defaulttop30], 'quart3':[svs_quart3_swe_defaulttop30]})
svs_dfs['Arctic']=pd.DataFrame({'mean':[svs_mean_swe_arctic], 'median':[svs_median_swe_arctic], 'quart1':[svs_quart1_swe_arctic], 'quart3':[svs_quart3_swe_arctic]})
svs_dfs['Arctic Top30']=pd.DataFrame({'mean':[svs_mean_swe_arctictop30], 'median':[svs_median_swe_arctictop30], 'quart1':[svs_quart1_swe_arctictop30], 'quart3':[svs_quart3_swe_arctictop30]})
svs_dfs['Arctic High Uncertainty']=pd.DataFrame({'mean':[svs_mean_swe_arctic], 'median':[svs_median_swe_arctic], 'quart1':[svs_quart1_swe_arctic], 'quart3':[svs_quart3_swe_arctic]})
svs_dfs['Arctic Four Observations']=pd.DataFrame({'mean':[svs_mean_swe_arctic], 'median':[svs_median_swe_arctic], 'quart1':[svs_quart1_swe_arctic], 'quart3':[svs_quart3_swe_arctic]})

### Calculating the Root-Mean-Square-Errors

In [ ]:
rmse_default = np.sqrt(np.mean((df_swe_default['Measure_median']-df_swe_default['Retrieved_median'])**2))
rmse_defaulttop30 = np.sqrt(np.mean((df_swe_defaulttop30['Measure_median']-df_swe_defaulttop30['Retrieved_median'])**2))
rmse_arctic = np.sqrt(np.mean((df_swe_arctic['Measure_median']-df_swe_arctic['Retrieved_median'])**2))
rmse_arctictop30 = np.sqrt(np.mean((df_swe_arctictop30['Measure_median']-df_swe_arctictop30['Retrieved_median'])**2))
rmse_arcticcond = np.sqrt(np.mean((df_swe_arcticcond['Measure_median']-df_swe_arcticcond['Retrieved_median'])**2))
rmse_arctic6 = np.sqrt(np.mean((df_swe_arctic6['Measure_median']-df_swe_arctic6['Retrieved_median'])**2))

In [ ]:
print(f'SWE RMSE Default Ensembles = ' + '{:.2f}'.format(rmse_default) + ' mm')
print(f'SWE RMSE Top 30 Default Ensembles = ' + '{:.2f}'.format(rmse_defaulttop30) + 'mm')
print(f'SWE RMSE Arctic Ensembles = ' + '{:.2f}'.format(rmse_arctic) + ' mm')
print(f'SWE RMSE Top30 Arctic Ensembles = ' + '{:.2f}'.format(rmse_arctictop30) + ' mm')
print(f'SWE RMSE Arctic Ensembles with larger uncertainty = ' + '{:.2f}'.format(rmse_arcticcond) + ' mm')
print(f'SWE RMSE Arctic Ensembles with larger uncertainty and four observations = ' + '{:.2f}'.format(rmse_arctic6) + ' mm')

SWE RMSE Default Ensembles = 27.61 mm  
SWE RMSE Top 30 Default Ensembles = 17.94mm  
SWE RMSE Arctic Ensembles = 20.88 mm  
SWE RMSE Top30 Arctic Ensembles = 21.17 mm  
SWE RMSE Arctic Ensembles with larger uncertainty = 18.68 mm  
SWE RMSE Arctic Ensembles with larger uncertainty and four observations = 15.80 mm

#### Calculating mean quartile deviation of posterior distributions

In [ ]:
std_default = np.mean((df_swe_default['Retrieved_quart3']-df_swe_default['Retrieved_quart1'])/2)
std_defaulttop30 = np.mean((df_swe_defaulttop30['Retrieved_quart3']-df_swe_defaulttop30['Retrieved_quart1'])/2)
std_arctic = np.mean((df_swe_arctic['Retrieved_quart3']-df_swe_arctic['Retrieved_quart1'])/2)
std_arctictop30 = np.mean((df_swe_arctictop30['Retrieved_quart3']-df_swe_arctictop30['Retrieved_quart1'])/2)
std_arcticcond = np.mean((df_swe_arcticcond['Retrieved_quart3']-df_swe_arcticcond['Retrieved_quart1'])/2)
std_arctic6 = np.mean((df_swe_arctic6['Retrieved_quart3']-df_swe_arctic6['Retrieved_quart1'])/2)

In [ ]:
print(f'SWE QD Default Ensembles = ' + '{:.2f}'.format(std_default) + ' mm')
print(f'SWE QD Top 30 Default Ensembles = ' + '{:.2f}'.format(std_defaulttop30) + 'mm')
print(f'SWE QD Arctic Ensembles = ' + '{:.2f}'.format(std_arctic) + ' mm')
print(f'SWE QD Top30 Arctic Ensembles = ' + '{:.2f}'.format(std_arctictop30) + ' mm')
print(f'SWE QD Arctic Ensembles with larger uncertainty = ' + '{:.2f}'.format(std_arcticcond) + ' mm')
print(f'SWE QD Arctic Ensembles with larger uncertainty and four observations = ' + '{:.2f}'.format(std_arctic6) + ' mm')

SWE QD Default Ensembles = 11.93 mm  
SWE QD Top 30 Default Ensembles = 6.86mm  
SWE QD Arctic Ensembles = 19.56 mm  
SWE QD Top30 Arctic Ensembles = 71.37 mm  
SWE QD Arctic Ensembles with larger uncertainty = 22.52 mm  
SWE QD Arctic Ensembles with larger uncertainty and four observations = 23.44 mm

In [ ]:
std_default = np.mean((df_swe_default['Retrieved_quart3']-df_swe_default['Retrieved_quart1'])/2/df_swe_default['Measure_median'])
std_defaulttop30 = np.mean((df_swe_defaulttop30['Retrieved_quart3']-df_swe_defaulttop30['Retrieved_quart1'])/2/df_swe_defaulttop30['Measure_median'])
std_arctic = np.mean((df_swe_arctic['Retrieved_quart3']-df_swe_arctic['Retrieved_quart1'])/2/df_swe_arctic['Measure_median'])
std_arctictop30 = np.mean((df_swe_arctictop30['Retrieved_quart3']-df_swe_arctictop30['Retrieved_quart1'])/2/df_swe_arctictop30['Measure_median'])
std_arcticcond = np.mean((df_swe_arcticcond['Retrieved_quart3']-df_swe_arcticcond['Retrieved_quart1'])/2/df_swe_arcticcond['Measure_median'])
std_arctic6 = np.mean((df_swe_arctic6['Retrieved_quart3']-df_swe_arctic6['Retrieved_quart1'])/2/df_swe_arctic6['Measure_median'])

In [ ]:
print(f'SWE QD Default Ensembles = ' + '{:.1f}'.format(std_default*100) + ' mm')
print(f'SWE QD Top 30 Default Ensembles = ' + '{:.1f}'.format(std_defaulttop30*100) + 'mm')
print(f'SWE QD Arctic Ensembles = ' + '{:.1f}'.format(std_arctic*100) + ' mm')
print(f'SWE QD Top30 Arctic Ensembles = ' + '{:.1f}'.format(std_arctictop30*100) + ' mm')
print(f'SWE QD Arctic Ensembles with larger uncertainty = ' + '{:.1f}'.format(std_arcticcond*100) + ' mm')
print(f'SWE QD Arctic Ensembles with larger uncertainty and four observations = ' + '{:.1f}'.format(std_arctic6*100) + ' mm')

SWE QD Default Ensembles = 12.9 mm  
SWE QD Top 30 Default Ensembles = 7.4mm  
SWE QD Arctic Ensembles = 21.1 mm  
SWE QD Top30 Arctic Ensembles = 20.2 mm  
SWE QD Arctic Ensembles with larger uncertainty = 24.2 mm  
SWE QD Arctic Ensembles with larger uncertainty and four observations = 25.2 mm

### Organizing the dataframes into a large dictionary

In [ ]:
swe_dfs = {}

swe_dfs['Default']=df_swe_default
swe_dfs['Default Top30']=df_swe_defaulttop30
swe_dfs['Arctic']=df_swe_arctic
swe_dfs['Arctic Top30']=df_swe_arctictop30
swe_dfs['Arctic High Uncertainty']=df_swe_arcticcond
swe_dfs['Arctic Four Observations']=df_swe_arctic6

### For discussion purposes only, not published, the R$^2$ is calculated here, with two different methods. The statistics did not prove to be significant to the results section

In [ ]:
from scipy.stats import linregress
from sklearn.metrics import r2_score

In [ ]:
m_def, b_def, r_def, p_def, std_err_def = linregress(df_swe_default['Measure_median'], df_swe_default['Retrieved_median'])
m_deftop30, b_deftop30, r_deftop30, p_deftop30, std_err_deftop30 = linregress(df_swe_defaulttop30['Measure_median'], df_swe_defaulttop30['Retrieved_median'])
m_arc, b_arc, r_arc, p_arc, std_err_arc = linregress(df_swe_arctic['Measure_median'], df_swe_arctic['Retrieved_median'])
m_arctop30, b_arctop30, r_arctop30, p_arctop30, std_err_arctop30 = linregress(df_swe_arctictop30['Measure_median'], df_swe_arctictop30['Retrieved_median'])
m_arcticcond, b_arcticcond, r_arcticcond, p_arcticcond, std_err_arcticcond = linregress(df_swe_arcticcond['Measure_median'], df_swe_arcticcond['Retrieved_median'])
m_arc6, b_arc6, r_arc6, p_arc6, std_err_arc6 = linregress(df_swe_arctic6['Measure_median'], df_swe_arctic6['Retrieved_median'])

In [ ]:
r2_def = r2_score(df_swe_default['Measure_median'], df_swe_default['Retrieved_median'])
r2_deftop30 = r2_score(df_swe_defaulttop30['Measure_median'], df_swe_defaulttop30['Retrieved_median'])
r2_arc = r2_score(df_swe_arctic['Measure_median'], df_swe_arctic['Retrieved_median'])
r2_arctop30 = r2_score(df_swe_arctictop30['Measure_median'], df_swe_arctictop30['Retrieved_median'])
r2_arcticcond = r2_score(df_swe_arcticcond['Measure_median'], df_swe_arcticcond['Retrieved_median'])
r2_arc6 = r2_score(df_swe_arctic6['Measure_median'], df_swe_arctic6['Retrieved_median'])

In [ ]:
print(f'SWE R^2 Default Ensembles = ' + '{:.2f}'.format(r_def**2))
print(f'SWE R^2 Top 30 Default Ensembles = ' + '{:.2f}'.format(r_deftop30**2))
print(f'SWE R^2 Arctic Ensembles = ' + '{:.2f}'.format(r_arc**2))
print(f'SWE R^2 Top30 Arctic Ensembles = ' + '{:.2f}'.format(r_arctop30**2))
print(f'SWE R^2 Arctic Ensembles with larger uncertainty = ' + '{:.2f}'.format(r_arcticcond**2))
print(f'SWE R^2 Arctic Ensembles with larger uncertainty and four observations = ' + '{:.2f}'.format(r_arc6**2))

SWE R$^2$ Default Ensembles = 0.01  
SWE R$^2$ Top 30 Default Ensembles = 0.05  
SWE R$^2$ Arctic Ensembles = 0.14  
SWE R$^2$ Top30 Arctic Ensembles = 0.11  
SWE R$^2$ Arctic Ensembles with larger uncertainty = 0.00  
SWE R$^2$ Arctic Ensembles with larger uncertainty and four observations = 0.13

In [ ]:
print(f'SWE R^2 Default Ensembles = ' + '{:.2f}'.format(r2_def))
print(f'SWE R^2 Top 30 Default Ensembles = ' + '{:.2f}'.format(r2_deftop30))
print(f'SWE R^2 Arctic Ensembles = ' + '{:.2f}'.format(r2_arc))
print(f'SWE R^2 Top30 Arctic Ensembles = ' + '{:.2f}'.format(r2_arctop30))
print(f'SWE R^2 Arctic Ensembles with larger uncertainty = ' + '{:.2f}'.format(r2_arcticcond))
print(f'SWE R^2 Arctic Ensembles with larger uncertainty and four observations = ' + '{:.2f}'.format(r2_arc6))

SWE R$^2$ Default Ensembles = -1.68  
SWE R$^2$ Top 30 Default Ensembles = -0.13  
SWE R$^2$ Arctic Ensembles = -0.54  
SWE R$^2$ Top30 Arctic Ensembles = -0.58  
SWE R$^2$ Arctic Ensembles with larger uncertainty = -0.23  
SWE R$^2$ Arctic Ensembles with larger uncertainty and four observations = 0.12  

In [ ]:
#Ensembles to compare
ensembles = ['Default','Default Top30']

fig, ax = plt.subplots(1,len(ensembles),figsize=(20,10))

for i in range(len(ensembles)):
  
    ax[i].fill_between([0,200],y1=[30,230],y2=[-30,170], color='grey', alpha=0.5, label='TSMM \nexpected \naccuracy')
    markers, caps, bars = ax[i].errorbar(swe_dfs[ensembles[i]]['Measure_median'],swe_dfs[ensembles[i]]['Retrieved_median'],
                                        xerr=[swe_dfs[ensembles[i]]['Measure_median']-swe_dfs[ensembles[i]]['Measure_quart1'],
                                              swe_dfs[ensembles[i]]['Measure_quart3']-swe_dfs[ensembles[i]]['Measure_median']],
                                        yerr=[swe_dfs[ensembles[i]]['Retrieved_median']-swe_dfs[ensembles[i]]['Retrieved_quart1'],
                                              swe_dfs[ensembles[i]]['Retrieved_quart3']-swe_dfs[ensembles[i]]['Retrieved_median']], 
                                        label='TVCExp18/19\nRetrieval',
                                        marker='o', linestyle='', color='k',capsize=5, linewidth=1)

    [bar.set_alpha(0.5) for bar in bars]
    [cap.set_alpha(0.5) for cap in caps]
    
    ax[i].axhline(svs_dfs[ensembles[i]]['median'].values, linestyle='--', color='red', linewidth=2, label='SVS-2 Initial\nSWE')
    ax[i].fill_between([0, 200], y1=[svs_dfs[ensembles[i]]['quart1'].values[0][0], svs_dfs[ensembles[i]]['quart1'].values[0][0]],
                                 y2=[svs_dfs[ensembles[i]]['quart3'].values[0][0], svs_dfs[ensembles[i]]['quart3'].values[0][0]], 
                       color='red', linewidth=1, alpha = 0.5, label='SVS-2 Initial\n$\sigma_{SWE}$');
    ax[i].plot([0,200],[0,200],'-',color='grey')
    ax[i].set_xlim(0,200);
    ax[i].set_ylim(0,200);
    ax[i].set_xlabel('Measured SWE (mm)');
    ax[i].text(75,175,'RMSE = \n{:2.1f} mm'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))));
    ax[i].text(75,160,'RMSE = \n{:2.1f}%'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))/
                                                swe_dfs[ensembles[i]]['Measure_median'].mean()*100));

ax[0].set_ylabel('Retrieved SWE (mm)');
ax[0].text(5,190,'a)');
ax[1].text(5,190,'b)');
ax[0].legend(loc=4);

fig.savefig('Figures/SWE_results_Def-DefTop30.png')

<center><img src="Figures/SWE_results_Def-DefTop30.png" Width="1000px"></center>

<center>Figure: Comparison of the MCMC outputs using priors from the a) 120 ensemble members and b) the top 30 ensemble members of the default SVS-2 version. Given that the SVS-2 runs were point scale, all surveyed sites start with the same prior distributions for a given test. The red line corresponds to the median value of SVS-2 and the red shading corresponds to the 1st and 3rd quartiles.</center>

In [ ]:
#Ensembles to compare
ensembles = ['Default','Arctic']

fig, ax = plt.subplots(1,len(ensembles),figsize=(20,10))

for i in range(len(ensembles)):
  
    ax[i].fill_between([0,200],y1=[30,230],y2=[-30,170], color='grey', alpha=0.5, label='TSMM \nexpected \naccuracy')
    markers, caps, bars = ax[i].errorbar(swe_dfs[ensembles[i]]['Measure_median'],swe_dfs[ensembles[i]]['Retrieved_median'],
                                        xerr=[swe_dfs[ensembles[i]]['Measure_median']-swe_dfs[ensembles[i]]['Measure_quart1'],
                                              swe_dfs[ensembles[i]]['Measure_quart3']-swe_dfs[ensembles[i]]['Measure_median']],
                                        yerr=[swe_dfs[ensembles[i]]['Retrieved_median']-swe_dfs[ensembles[i]]['Retrieved_quart1'],
                                              swe_dfs[ensembles[i]]['Retrieved_quart3']-swe_dfs[ensembles[i]]['Retrieved_median']], 
                                        label='TVCExp18/19\nRetrieval',
                                        marker='o', linestyle='', color='k',capsize=5, linewidth=1)

    [bar.set_alpha(0.5) for bar in bars]
    [cap.set_alpha(0.5) for cap in caps]

    ax[i].axhline(svs_dfs[ensembles[i]]['median'].values, linestyle='--', color='red', linewidth=2, label='SVS-2 Initial\nSWE')
    ax[i].fill_between([0, 200], y1=[svs_dfs[ensembles[i]]['quart1'].values[0][0], svs_dfs[ensembles[i]]['quart1'].values[0][0]],
                                 y2=[svs_dfs[ensembles[i]]['quart3'].values[0][0], svs_dfs[ensembles[i]]['quart3'].values[0][0]], 
                       color='red', linewidth=1, alpha = 0.5, label='SVS-2 Initial\n$\sigma_{SWE}$');
    ax[i].plot([0,200],[0,200],'-',color='grey')
    ax[i].set_xlim(0,200);
    ax[i].set_ylim(0,200);
    ax[i].set_xlabel('Measured SWE (mm)');
    ax[i].text(75,175,'RMSE = \n{:2.1f} mm'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))));
    ax[i].text(75,160,'RMSE = \n{:2.1f}%'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))/
                                                swe_dfs[ensembles[i]]['Measure_median'].mean()*100));

ax[0].set_ylabel('Retrieved SWE (mm)');
ax[0].text(5,190,'a)');
ax[1].text(5,190,'b)');
ax[0].legend(loc=4);

fig.savefig('Figures/f07.png')

<center><img src="Figures/f07.png" Width="1000px"></center>

<center>Figure: Comparison of the MCMC outputs using priors from the 120 ensemble members of a) the default, and b) the Arctic versions of SVS-2. Given that the SVS-2 runs were point scale, all surveyed sites start with the same prior distributions for a given test. The red line corresponds to the median value of SVS-2 and the red shading corresponds to the 1st and 3rd quartiles.</center>

In [ ]:
#Ensembles to compare
ensembles = ['Default Top30','Arctic']

fig, ax = plt.subplots(1,len(ensembles),figsize=(20,10))

for i in range(len(ensembles)):
  
    ax[i].fill_between([0,200],y1=[30,230],y2=[-30,170], color='grey', alpha=0.5, label='TSMM \nexpected \naccuracy')
    markers, caps, bars = ax[i].errorbar(swe_dfs[ensembles[i]]['Measure_median'],swe_dfs[ensembles[i]]['Retrieved_median'],
                                        xerr=[swe_dfs[ensembles[i]]['Measure_median']-swe_dfs[ensembles[i]]['Measure_quart1'],
                                              swe_dfs[ensembles[i]]['Measure_quart3']-swe_dfs[ensembles[i]]['Measure_median']],
                                        yerr=[swe_dfs[ensembles[i]]['Retrieved_median']-swe_dfs[ensembles[i]]['Retrieved_quart1'],
                                              swe_dfs[ensembles[i]]['Retrieved_quart3']-swe_dfs[ensembles[i]]['Retrieved_median']], 
                                        label='TVCExp18/19\nRetrieval',
                                        marker='o', linestyle='', color='k',capsize=5, linewidth=1)

    [bar.set_alpha(0.5) for bar in bars]
    [cap.set_alpha(0.5) for cap in caps]

    ax[i].axhline(svs_dfs[ensembles[i]]['median'].values, linestyle='--', color='red', linewidth=2, label='SVS-2 Initial\nSWE')
    ax[i].fill_between([0, 200], y1=[svs_dfs[ensembles[i]]['quart1'].values[0][0], svs_dfs[ensembles[i]]['quart1'].values[0][0]],
                                 y2=[svs_dfs[ensembles[i]]['quart3'].values[0][0], svs_dfs[ensembles[i]]['quart3'].values[0][0]], 
                       color='red', linewidth=1, alpha = 0.5, label='SVS-2 Initial\n$\sigma_{SWE}$');
    ax[i].plot([0,200],[0,200],'-',color='grey')
    ax[i].set_xlim(0,200);
    ax[i].set_ylim(0,200);
    ax[i].set_xlabel('Measured SWE (mm)');
    ax[i].text(75,175,'RMSE = \n{:2.1f} mm'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))));
    ax[i].text(75,160,'RMSE = \n{:2.1f}%'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))/
                                                swe_dfs[ensembles[i]]['Measure_median'].mean()*100));
ax[0].set_ylabel('Retrieved SWE (mm)');
ax[0].text(5,190,'a)');
ax[1].text(5,190,'b)');
ax[0].legend(loc=4);

fig.savefig('Figures/SWE_results_DefTop30-Arc.png')

<center><img src="Figures/SWE_results_DefTop30-Arc.png" Width="1000px"></center>

<center>Figure: Comparison of the MCMC outputs using priors from a) the top 30 ensemble members of the default version of SVS-2, and b) all ensemble members of the Arctic version of SVS-2. Given that the SVS-2 runs were point scale, all surveyed sites start with the same prior distributions for a given test. The red line corresponds to the median value of SVS-2 and the red shading corresponds to the 1st and 3rd quartiles.</center>

In [ ]:
#Ensembles to compare
ensembles = ['Default Top30','Arctic Top30']

fig, ax = plt.subplots(1,len(ensembles),figsize=(20,10))

for i in range(len(ensembles)):
  
    ax[i].fill_between([0,200],y1=[30,230],y2=[-30,170], color='grey', alpha=0.5, label='TSMM \nexpected \naccuracy')
    markers, caps, bars = ax[i].errorbar(swe_dfs[ensembles[i]]['Measure_median'],swe_dfs[ensembles[i]]['Retrieved_median'],
                                        xerr=[swe_dfs[ensembles[i]]['Measure_median']-swe_dfs[ensembles[i]]['Measure_quart1'],
                                              swe_dfs[ensembles[i]]['Measure_quart3']-swe_dfs[ensembles[i]]['Measure_median']],
                                        yerr=[swe_dfs[ensembles[i]]['Retrieved_median']-swe_dfs[ensembles[i]]['Retrieved_quart1'],
                                              swe_dfs[ensembles[i]]['Retrieved_quart3']-swe_dfs[ensembles[i]]['Retrieved_median']], 
                                        label='TVCExp18/19\nRetrieval',
                                        marker='o', linestyle='', color='k',capsize=5, linewidth=1)

    [bar.set_alpha(0.5) for bar in bars]
    [cap.set_alpha(0.5) for cap in caps]

    ax[i].axhline(svs_dfs[ensembles[i]]['median'].values, linestyle='--', color='red', linewidth=2, label='SVS-2 Initial\nSWE')
    ax[i].fill_between([0, 200], y1=[svs_dfs[ensembles[i]]['quart1'].values[0][0], svs_dfs[ensembles[i]]['quart1'].values[0][0]],
                                 y2=[svs_dfs[ensembles[i]]['quart3'].values[0][0], svs_dfs[ensembles[i]]['quart3'].values[0][0]], 
                       color='red', linewidth=1, alpha = 0.5, label='SVS-2 Initial\n$\sigma_{SWE}$');
    ax[i].plot([0,200],[0,200],'-',color='grey')
    ax[i].set_xlim(0,200);
    ax[i].set_ylim(0,200);
    ax[i].set_xlabel('Measured SWE (mm)');
    ax[i].text(75,175,'RMSE = \n{:2.1f} mm'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))));
    ax[i].text(75,160,'RMSE = \n{:2.1f}%'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))/
                                                swe_dfs[ensembles[i]]['Measure_median'].mean()*100));
ax[0].set_ylabel('Retrieved SWE (mm)');
ax[0].text(5,190,'a)');
ax[1].text(5,190,'b)');
ax[0].legend(loc=4);

fig.savefig('Figures/f08.png')

<center><img src="Figures/f08.png" Width="1000px"></center>

<center>Figure: Comparison of the MCMC outputs using priors from the top 30 ensemble members of a) the default version of SVS-2, and b) the Arctic version of SVS-2. Given that the SVS-2 runs were point scale, all surveyed sites start with the same prior distributions for a given test. The red line corresponds to the median value of SVS-2 and the red shading corresponds to the 1st and 3rd quartiles.</center>

In [ ]:
#Ensembles to compare
ensembles = ['Arctic', 'Arctic Top30']

fig, ax = plt.subplots(1,len(ensembles),figsize=(20,10))

for i in range(len(ensembles)):
  
    ax[i].fill_between([0,200],y1=[30,230],y2=[-30,170], color='grey', alpha=0.5, label='TSMM \nexpected \naccuracy')
    markers, caps, bars = ax[i].errorbar(swe_dfs[ensembles[i]]['Measure_median'],swe_dfs[ensembles[i]]['Retrieved_median'],
                                        xerr=[swe_dfs[ensembles[i]]['Measure_median']-swe_dfs[ensembles[i]]['Measure_quart1'],
                                              swe_dfs[ensembles[i]]['Measure_quart3']-swe_dfs[ensembles[i]]['Measure_median']],
                                        yerr=[swe_dfs[ensembles[i]]['Retrieved_median']-swe_dfs[ensembles[i]]['Retrieved_quart1'],
                                              swe_dfs[ensembles[i]]['Retrieved_quart3']-swe_dfs[ensembles[i]]['Retrieved_median']], 
                                        label='TVCExp18/19\nRetrieval',
                                        marker='o', linestyle='', color='k',capsize=5, linewidth=1)

    [bar.set_alpha(0.5) for bar in bars]
    [cap.set_alpha(0.5) for cap in caps]

    ax[i].axhline(svs_dfs[ensembles[i]]['median'].values, linestyle='--', color='red', linewidth=2, label='SVS-2 Initial\nSWE')
    ax[i].fill_between([0, 200], y1=[svs_dfs[ensembles[i]]['quart1'].values[0][0], svs_dfs[ensembles[i]]['quart1'].values[0][0]],
                                 y2=[svs_dfs[ensembles[i]]['quart3'].values[0][0], svs_dfs[ensembles[i]]['quart3'].values[0][0]], 
                       color='red', linewidth=1, alpha = 0.5, label='SVS-2 Initial\n$\sigma_{SWE}$');
    ax[i].plot([0,200],[0,200],'-',color='grey')
    ax[i].set_xlim(0,200);
    ax[i].set_ylim(0,200);
    ax[i].set_xlabel('Measured SWE (mm)');
    ax[i].text(75,175,'RMSE = \n{:2.1f} mm'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))));
    ax[i].text(75,160,'RMSE = \n{:2.1f}%'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))/
                                                swe_dfs[ensembles[i]]['Measure_median'].mean()*100));
ax[0].set_ylabel('Retrieved SWE (mm)');
ax[0].text(5,190,'a)');
ax[1].text(5,190,'b)');
ax[0].legend(loc=4);

fig.savefig('Figures/SWE_results_Arc-ArcTop30.png')

<center><img src="Figures/SWE_results_Arc-ArcTop30.png" Width="1000px"></center>

<center>Figure: Comparison of the MCMC outputs using priors from a) all the ensemble members, and b) the top 30 ensemble members of the Arctic version of SVS-2. Given that the SVS-2 runs were point scale, all surveyed sites start with the same prior distributions for a given test. The red line corresponds to the median value of SVS-2 and the red shading corresponds to the 1st and 3rd quartiles.</center>

In [ ]:
#Ensembles to compare
ensembles = ['Arctic','Arctic High Uncertainty']

fig, ax = plt.subplots(1,len(ensembles),figsize=(20,10))

for i in range(len(ensembles)):
  
    ax[i].fill_between([0,200],y1=[30,230],y2=[-30,170], color='grey', alpha=0.5, label='TSMM \nexpected \naccuracy')
    markers, caps, bars = ax[i].errorbar(swe_dfs[ensembles[i]]['Measure_median'],swe_dfs[ensembles[i]]['Retrieved_median'],
                                        xerr=[swe_dfs[ensembles[i]]['Measure_median']-swe_dfs[ensembles[i]]['Measure_quart1'],
                                              swe_dfs[ensembles[i]]['Measure_quart3']-swe_dfs[ensembles[i]]['Measure_median']],
                                        yerr=[swe_dfs[ensembles[i]]['Retrieved_median']-swe_dfs[ensembles[i]]['Retrieved_quart1'],
                                              swe_dfs[ensembles[i]]['Retrieved_quart3']-swe_dfs[ensembles[i]]['Retrieved_median']], 
                                        label='TVCExp18/19\nRetrieval',
                                        marker='o', linestyle='', color='k',capsize=5, linewidth=1)

    [bar.set_alpha(0.5) for bar in bars]
    [cap.set_alpha(0.5) for cap in caps]

    ax[i].axhline(svs_dfs[ensembles[i]]['median'].values, linestyle='--', color='red', linewidth=2, label='SVS-2 Initial\nSWE')
    ax[i].fill_between([0, 200], y1=[svs_dfs[ensembles[i]]['quart1'].values[0][0], svs_dfs[ensembles[i]]['quart1'].values[0][0]],
                                 y2=[svs_dfs[ensembles[i]]['quart3'].values[0][0], svs_dfs[ensembles[i]]['quart3'].values[0][0]], 
                       color='red', linewidth=1, alpha = 0.5, label='SVS-2 Initial\n$\sigma_{SWE}$');
    ax[i].plot([0,200],[0,200],'-',color='grey')
    ax[i].set_xlim(0,200);
    ax[i].set_ylim(0,200);
    ax[i].set_xlabel('Measured SWE (mm)');
    ax[i].text(75,175,'RMSE = \n{:2.1f} mm'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))));
    ax[i].text(75,160,'RMSE = \n{:2.1f}%'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))/
                                                swe_dfs[ensembles[i]]['Measure_median'].mean()*100));
    ax[i].set_title(ensembles[i] + ' SVS-2 version');

ax[0].set_ylabel('Retrieved SWE (mm)');
ax[0].text(5,190,'a)');
ax[1].text(5,190,'b)');
ax[0].legend(loc=4);

fig.savefig('Figures/SWE_results_Arc-ArcSigSSA.png')

<center><img src="Figures/SWE_results_Arc-ArcSigSSA.png" Width="1000px"></center>

<center>Figure: Comparison of the MCMC outputs using priors from all ensemble members of the Arctic verison of SVS-2 with a) the modeled uncertainty on SSA, and b) three times the modeled uncertainty on SSA. Given that the SVS-2 runs were point scale, all surveyed sites start with the same prior distributions for a given test. The red line corresponds to the median value of SVS-2 and the red shading corresponds to the 1st and 3rd quartiles.</center>

In [ ]:
#Ensembles to compare
ensembles = ['Arctic High Uncertainty','Arctic Four Observations']

fig, ax = plt.subplots(1,len(ensembles),figsize=(20,10))

for i in range(len(ensembles)):
  
    ax[i].fill_between([0,200],y1=[30,230],y2=[-30,170], color='grey', alpha=0.5, label='TSMM \nexpected \naccuracy')
    markers, caps, bars = ax[i].errorbar(swe_dfs[ensembles[i]]['Measure_median'],swe_dfs[ensembles[i]]['Retrieved_median'],
                                        xerr=[swe_dfs[ensembles[i]]['Measure_median']-swe_dfs[ensembles[i]]['Measure_quart1'],
                                              swe_dfs[ensembles[i]]['Measure_quart3']-swe_dfs[ensembles[i]]['Measure_median']],
                                        yerr=[swe_dfs[ensembles[i]]['Retrieved_median']-swe_dfs[ensembles[i]]['Retrieved_quart1'],
                                              swe_dfs[ensembles[i]]['Retrieved_quart3']-swe_dfs[ensembles[i]]['Retrieved_median']], 
                                        label='TVCExp18/19\nRetrieval',
                                        marker='o', linestyle='', color='k',capsize=5, linewidth=1)

    [bar.set_alpha(0.5) for bar in bars]
    [cap.set_alpha(0.5) for cap in caps]
    
    ax[i].axhline(svs_dfs[ensembles[i]]['median'].values, linestyle='--', color='red', linewidth=2, label='SVS-2 Initial\nSWE')
    ax[i].fill_between([0, 200], y1=[svs_dfs[ensembles[i]]['quart1'].values[0][0], svs_dfs[ensembles[i]]['quart1'].values[0][0]],
                                 y2=[svs_dfs[ensembles[i]]['quart3'].values[0][0], svs_dfs[ensembles[i]]['quart3'].values[0][0]], 
                       color='red', linewidth=1, alpha = 0.5, label='SVS-2 Initial\n$\sigma_{SWE}$');
    ax[i].plot([0,200],[0,200],'-',color='grey')
    ax[i].set_xlim(0,200);
    ax[i].set_ylim(0,200);
    ax[i].set_xlabel('Measured SWE (mm)');
    ax[i].text(75,175,'RMSE = \n{:2.1f} mm'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))));
    ax[i].text(75,160,'RMSE = \n{:2.1f}%'.format(np.sqrt(np.mean((swe_dfs[ensembles[i]]['Measure_median']-
                                                                   swe_dfs[ensembles[i]]['Retrieved_median'])**2))/
                                                swe_dfs[ensembles[i]]['Measure_median'].mean()*100));

ax[0].set_ylabel('Retrieved SWE (mm)');
ax[0].text(5,190,'a)');
ax[1].text(5,190,'b)');
ax[0].legend(loc=4);

fig.savefig('Figures/f09.png')

<center><img src="Figures/f09.png" Width="1000px"></center>

<center>Figure: Comparison of the MCMC outputs using priors from all ensemble members of the Arctic verison of SVS-2 with three times the uncertainty on SSA and a) one observation around 35$^o$ incidence angle, and b) four observations randomly selected within the TSMM incidence angle range (25$^o$<$\theta$<55$^o$). Given that the SVS-2 runs were point scale, all surveyed sites start with the same prior distributions for a given test. The red line corresponds to the median value of SVS-2 and the red shading corresponds to the 1st and 3rd quartiles.</center>

# This section generates a dataframe for the 10 000 iterations test and generate the figure published in [Montpetit et al. (Preprint)](TBD)

In [ ]:
if os.path.isfile('Data/MCMC_Outputs/SWE_Arctic_draws.csv'):

    df_swe_arctic_draws=pd.read_csv('Data/MCMC_Outputs/SWE_Arctic_draws.csv')

else:

    files = [f for f in os.listdir('Data/MCMC_Outputs') if f.endswith('4obs_Arctic_WSRMSE_conditions.nc')]
    
    df_swe_arctic_draws = pd.DataFrame({'Site':[]})
    
    for file in files:
    
        site=file.split('_')[2]
    
        if site not in df_swe_arctic_draws.Site.values:
    
            for draw in range(100,10100,100):
        
                idata=az.from_netcdf('Data/MCMC_Outputs/' + file)
                idata = idata.sel(draw=slice(draw))
                
                idata.posterior = idata.posterior.assign(SWE=(idata.posterior['Density_R']*idata.posterior['Thickness_R']+
                                                              idata.posterior['Density_H']*idata.posterior['Thickness_H']))
                df_swe_arctic_draws = pd.concat([df_swe_arctic_draws,
                                        pd.DataFrame({'Site':[site],'Draw':[draw],
                                                      'Measure_median':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['median'].values],
                                                      'Measure_std':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['std'].values],
                                                      'Measure_quart1':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart1'].values],
                                                      'Measure_quart3':[snowpit_stats.sel(site=site, property='swe', grain_type='R')['quart3'].values],
                                                      'Retrieved_median':[idata.posterior['SWE'].median().values],
                                                      'Retrieved_std':[idata.posterior['SWE'].std().values],
                                                      'Retrieved_quart1':[idata.posterior['SWE'].quantile(0.25).values],
                                                      'Retrieved_quart3':[idata.posterior['SWE'].quantile(0.75).values],
                                                      'rhat':[az.rhat(idata)['SWE'].values],
                                                      'ess':[az.ess(idata)['SWE'].values]})],
                                       ignore_index=True)
    
    df_swe_arctic_draws['bias'] = df_swe_arctic_draws.Measure_median-df_swe_arctic_draws.Retrieved_median
    df_swe_arctic_draws.to_csv('Data/MCMC_Outputs/SWE_Arctic_draws.csv')
df_swe_arctic_draws.sort_values('Site')

In [ ]:
fig, ax = plt.subplots(3,1,figsize=(7,14))

#R_hat subplot

y1=df_swe_arctic_draws.groupby('Draw').rhat.min().values.astype(np.float32)
y2=df_swe_arctic_draws.groupby('Draw').rhat.max().values.astype(np.float32)

ax[0].plot(range(100,10100,100), 
         df_swe_arctic_draws.groupby('Draw').rhat.agg(lambda x: np.median(x)),'k', label='Mean $\\hat{R}$')
ax[0].fill_between(range(100,10100,100),
               y1=y2,
               y2=y1, color='k', alpha=0.5, label='Min/Max $\\hat{R}$ \nrange')
ax[0].axhline(1.01,linestyle='--',color='r', label='Acceptable $\\hat{R}$ \nthreshold')
ax[0].set_ylabel('$\\hat{R}$');
ax[0].set_xlim(0,10000);
ax[0].set_xticklabels('');
ax[0].legend();

#ESS subplot

y1=df_swe_arctic_draws.groupby('Draw').ess.min().values.astype(np.float32)
y2=df_swe_arctic_draws.groupby('Draw').ess.max().values.astype(np.float32)

ax[1].plot(range(100,10100,100), 
         df_swe_arctic_draws.groupby('Draw').ess.mean(), 'k', label='Mean ESS')
ax[1].fill_between(range(100,10100,100),
               y1=y2,
               y2=y1, color='k', alpha=0.5, label='Min/Max \nESS range')
ax[1].axhline(100,linestyle='--',color='r', label='Acceptable ESS \nthreshold')
ax[1].set_ylabel('ESS');
ax[1].set_xlim(0,10000);
ax[1].set_xticklabels('');
ax[1].set_ylim(0,3000);
ax[1].legend();

#RMSE subplot

ax[2].plot(range(100,10100,100), 
         np.sqrt(df_swe_arctic_draws.groupby('Draw').bias.apply(lambda x: x**2).groupby(level=0).mean().values.astype(np.float32)),
          color='k')
ax[2].set_ylabel('RMSE (mm)')
ax[2].set_xlabel('Number of iterations')
ax[2].set_xlim(0,10000)

fig.savefig('Figures/f05.png')

<center><img src="Figures/f05.png" Width="500px"></center>

<center>Figure: Evolution of a) the inter-chain convergence ($\hat{R}$), b) the Equivalent Sample Size (ESS), and c) the root-mean-square-error (RMSE), over 10 000 iterations. The published, acceptable threshold for the $\hat{R}$ and ESS are also shown.</center>

## The next cell generates the MCMC trace plot, showing the prior/posterior distributions (left) and the evolution of the sampled variables (right) for the runs using the default SVS-2 versionand inter-layer constraints

In [ ]:
var_names = ['Thickness_R','Thickness_H',
             'Density_R','Density_H',
             'SSA_R','SSA_H',
             'SWE','Sigma']
variables = ['thickness','thickness',
            'density','density',
            'ssa','ssa',
            'swe']
gtypes=['R','H','R','H','R','H','R','H']
xmins=[0,0,100,100,5,5,0,0]
xmaxs=[1,1,450,450,50,50,300,2.5]

for site in list(df_swe_arctic6.Site):

    idata_site = az.from_netcdf('Data/MCMC_Output_'+site+'_1obs_Default.nc')
    idata_site.posterior = idata_site.posterior.assign(SWE=(idata_site.posterior['Density_R']*idata_site.posterior['Thickness_R']+
                                                          idata_site.posterior['Density_H']*idata_site.posterior['Thickness_H']))
    idata_site.prior = idata_site.prior.assign(SWE=(idata_site.prior['Density_R']*idata_site.prior['Thickness_R']+
                                                          idata_site.prior['Density_H']*idata_site.prior['Thickness_H']))
    
    fig, ax = plt.subplots(8,2,figsize=(10,20))
    
    for i in range(len(var_names)):
    
        idata_site.prior[var_names[i]].mean(dim='chain').to_dataframe().plot(ax=ax[i,0],kind='density',color='grey',
                                                                             label='Prior',legend=None,alpha=0.7);
        idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().plot(ax=ax[i,0],kind='density',color='k',
                                                                                 label='Posterior',legend=None);
        idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().plot(ax=ax[i,1],color='k',legend=None);
    
        y1=idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).min(dim='chain').values
        y2=idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).max(dim='chain').values
        ax[i,1].fill_between(range(5000),y1=y2,y2=y1,color='k',alpha=0.5)
        
        if i<7:
    
            x=np.linspace(xmins[i],xmaxs[i],200)
            ax[i,0].plot(x, norm.pdf(x,loc=snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['median'].values,
                               scale=snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['std'].values), 
                       color='cyan', linewidth=2,alpha=0.7)
            
        ax[i,0].set_xlabel(var_names[i]);
        ax[i,0].set_ylabel('');
        ax[i,1].set_ylabel(var_names[i]);
        ax[i,1].set_xlabel('');
        ax[i,0].yaxis.set_major_formatter(FormatStrFormatter('%0.2f'));
    
    ax[0,0].set_xlim(0,1)
    ax[1,0].set_xlim(0,1)
    ax[2,0].set_xlim(100,450)
    ax[3,0].set_xlim(100,450)
    ax[4,0].set_xlim(5,50)
    ax[5,0].set_xlim(5,50)
    ax[6,0].set_xlim(0,300)
    ax[7,0].set_xlim(0,2.5)
    
    ax[3,0].set_ylabel('PDF')
    ax[7,1].set_xlabel('Iteration')
    ax[0,0].legend(['Prior','Posterior','Snowpits'],loc=1);
    fig.savefig('Figures/TracePlot_'+site+'_Default.png')

## The next cell generates the MCMC trace plot, showing the prior/posterior distributions (left) and the evolution of the sampled variables (right) for the runs using the Arctic SVS-2 version with three times the uncertainty on the SSA priors, four observations, and inter-layer constraints

In [ ]:
var_names = ['Thickness_R','Thickness_H',
             'Density_R','Density_H',
             'SSA_R','SSA_H',
             'SWE','Sigma']
variables = ['thickness','thickness',
            'density','density',
            'ssa','ssa',
            'swe']
gtypes=['R','H','R','H','R','H','R','H']
xmins=[0,0,100,100,5,5,0,0]
xmaxs=[1,1,450,450,50,50,300,2.5]

for site in list(df_swe_arctic6.Site):

    idata_site = az.from_netcdf('Data/MCMC_Outputs/MCMC_Output_'+site+'_4obs_Arctic_WSRMSE_conditions.nc')
    idata_site.posterior = idata_site.posterior.assign(SWE=(idata_site.posterior['Density_R']*idata_site.posterior['Thickness_R']+
                                                          idata_site.posterior['Density_H']*idata_site.posterior['Thickness_H']))
    idata_site.prior = idata_site.prior.assign(SWE=(idata_site.prior['Density_R']*idata_site.prior['Thickness_R']+
                                                          idata_site.prior['Density_H']*idata_site.prior['Thickness_H']))
    
    fig, ax = plt.subplots(8,2,figsize=(10,20))
    
    for i in range(len(var_names)):
    
        idata_site.prior[var_names[i]].mean(dim='chain').to_dataframe().plot(ax=ax[i,0],kind='density',color='grey',
                                                                             label='Prior',legend=None,alpha=0.7);
        idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().plot(ax=ax[i,0],kind='density',color='k',
                                                                                 label='Posterior',legend=None);
        idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().plot(ax=ax[i,1],color='k',legend=None);
    
        y1=idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).min(dim='chain').values
        y2=idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).max(dim='chain').values
        ax[i,1].fill_between(range(5000),y1=y2,y2=y1,color='k',alpha=0.5)
        
        if i<7:
    
            x=np.linspace(xmins[i],xmaxs[i],200)
            ax[i,0].plot(x, norm.pdf(x,loc=snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['median'].values,
                               scale=snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['std'].values), 
                       color='cyan', linewidth=2,alpha=0.7)
            
        ax[i,0].set_xlabel(var_names[i]);
        ax[i,0].set_ylabel('');
        ax[i,1].set_ylabel(var_names[i]);
        ax[i,1].set_xlabel('');
        ax[i,0].yaxis.set_major_formatter(FormatStrFormatter('%0.2f'));
    
    ax[0,0].set_xlim(0,1)
    ax[1,0].set_xlim(0,1)
    ax[2,0].set_xlim(100,450)
    ax[3,0].set_xlim(100,450)
    ax[4,0].set_xlim(5,50)
    ax[5,0].set_xlim(5,50)
    ax[6,0].set_xlim(0,300)
    ax[7,0].set_xlim(0,2.5)
    
    ax[3,0].set_ylabel('PDF')
    ax[7,1].set_xlabel('Iteration')
    ax[0,0].legend(['Prior','Posterior','Snowpits'],loc=1);
    fig.savefig('Figures/TracePlot_'+site+'_Final.png')

## The next cell generates the MCMC trace plot, showing the prior/posterior distributions (left) and the evolution of the sampled variables (right) for the runs using the Arctic SVS-2 version with three times the uncertainty on the SSA priors, four observations, and without inter-layer constraints

In [ ]:
import glob

In [ ]:
var_names = ['Thickness_R','Thickness_H',
             'Density_R','Density_H',
             'SSA_R','SSA_H',
             'SWE','Sigma']
variables = ['thickness','thickness',
            'density','density',
            'ssa','ssa',
            'swe']
gtypes=['R','H','R','H','R','H','R','H']
xmins=[0,0,100,100,5,5,0,0]
xmaxs=[1,1,450,450,50,50,300,2.5]

for site in list(df_swe_arctic6.Site):

    filepaths = glob.glob('Data/MCMC_Outputs/MCMC_Output_'+site+'_*_4obs_Arctic.nc')
    idata_site = az.from_netcdf(filepaths[0])
    idata_site.posterior = idata_site.posterior.assign(SWE=(idata_site.posterior['Density_R']*idata_site.posterior['Thickness_R']+
                                                          idata_site.posterior['Density_H']*idata_site.posterior['Thickness_H']))
    idata_site.prior = idata_site.prior.assign(SWE=(idata_site.prior['Density_R']*idata_site.prior['Thickness_R']+
                                                          idata_site.prior['Density_H']*idata_site.prior['Thickness_H']))
    
    fig, ax = plt.subplots(8,2,figsize=(10,20))
    
    for i in range(len(var_names)):
    
        idata_site.prior[var_names[i]].mean(dim='chain').to_dataframe().plot(ax=ax[i,0],kind='density',color='grey',
                                                                             label='Prior',legend=None,alpha=0.7);
        idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().plot(ax=ax[i,0],kind='density',color='k',
                                                                                 label='Posterior',legend=None);
        idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().plot(ax=ax[i,1],color='k',legend=None);
    
        y1=idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).min(dim='chain').values
        y2=idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).max(dim='chain').values
        ax[i,1].fill_between(range(5000),y1=y2,y2=y1,color='k',alpha=0.5)
        
        if i<7:
    
            x=np.linspace(xmins[i],xmaxs[i],200)
            ax[i,0].plot(x, norm.pdf(x,loc=snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['median'].values,
                               scale=snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['std'].values), 
                       color='cyan', linewidth=2,alpha=0.7)
            
        ax[i,0].set_xlabel(var_names[i]);
        ax[i,0].set_ylabel('');
        ax[i,1].set_ylabel(var_names[i]);
        ax[i,1].set_xlabel('');
        ax[i,0].yaxis.set_major_formatter(FormatStrFormatter('%0.2f'));
    
    ax[0,0].set_xlim(0,1)
    ax[1,0].set_xlim(0,1)
    ax[2,0].set_xlim(100,450)
    ax[3,0].set_xlim(100,450)
    ax[4,0].set_xlim(5,50)
    ax[5,0].set_xlim(5,50)
    ax[6,0].set_xlim(0,300)
    ax[7,0].set_xlim(0,2.5)
    
    ax[3,0].set_ylabel('PDF')
    ax[7,1].set_xlabel('Iteration')
    ax[0,0].legend(['Prior','Posterior','Snowpits'],loc=1);
    fig.savefig('Figures/TracePlot_'+site+'Free.png')

# This next section calculates the statistics between the prior and posterior distributions.

In [ ]:
idata_default = az.from_netcdf('Data/MCMC_Output_SM02_1obs_Default.nc')
idata_final = az.from_netcdf('Data/MCMC_Outputs/MCMC_Output_SM02_4obs_Arctic_WSRMSE_conditions.nc')
filepaths = glob.glob('Data/MCMC_Outputs/MCMC_Output_SM02_*_4obs_Arctic.nc')
idata_free = az.from_netcdf(filepaths[0])

In [ ]:
idata_default.posterior = idata_default.posterior.assign(SWE=(idata_default.posterior['Density_R']*idata_default.posterior['Thickness_R']+
                                                      idata_default.posterior['Density_H']*idata_default.posterior['Thickness_H']))
idata_default.prior = idata_default.prior.assign(SWE=(idata_default.prior['Density_R']*idata_default.prior['Thickness_R']+
                                                      idata_default.prior['Density_H']*idata_default.prior['Thickness_H']))

idata_final.posterior = idata_final.posterior.assign(SWE=(idata_final.posterior['Density_R']*idata_final.posterior['Thickness_R']+
                                                      idata_final.posterior['Density_H']*idata_final.posterior['Thickness_H']))
idata_final.prior = idata_final.prior.assign(SWE=(idata_final.prior['Density_R']*idata_final.prior['Thickness_R']+
                                                      idata_final.prior['Density_H']*idata_final.prior['Thickness_H']))

idata_free.posterior = idata_free.posterior.assign(SWE=(idata_free.posterior['Density_R']*idata_free.posterior['Thickness_R']+
                                                      idata_free.posterior['Density_H']*idata_free.posterior['Thickness_H']))
idata_free.prior = idata_free.prior.assign(SWE=(idata_free.prior['Density_R']*idata_free.prior['Thickness_R']+
                                                      idata_free.prior['Density_H']*idata_free.prior['Thickness_H']))

# Posterior vs Prior differences Default SVS-2

In [ ]:
var_names = ['Thickness_R','Thickness_H',
             'Density_R','Density_H',
             'SSA_R','SSA_H',
             'SWE','Sigma']
variables = ['thickness','thickness',
            'density','density',
            'ssa','ssa',
            'swe','sigma']
gtypes=['R','H','R','H','R','H','R','H']
units=['m','m','kg m-3','kg m-3','m2 kg-1','m2 kg-1','mm','dB']

for i in range(len(variables)):

    print(var_names[i] + ' : ' + '{:.3f}'.format((idata_default.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median() - 
      idata_default.prior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median()).values[0]) + ' ' + units[i])

Thickness_R : -0.144 m  
Thickness_H : 0.038 m  
Density_R : 4.917 kg m$^{-3}$  
Density_H : -0.126 kg m$^{-3}$  
SSA_R : 1.701 m$^2$ kg$^{-1}$  
SSA_H : 0.693 m$^2$ kg$^{-1}$  
SWE : -24.483 mm  
Sigma : -0.000 dB  

In [ ]:
for i in range(len(variables)):

    print(var_names[i] + ' : ' + '{:.3f}'.format(idata_default.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std().values[0]) + ' - '
                                 '{:.3f}'.format(idata_default.prior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std().values[0]) + ' = '
                                                  '{:.3f}'.format((idata_default.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std() - 
      idata_default.prior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std()).values[0]) + ' ' + units[i])

Thickness_R : 0.020 - 0.080 = -0.060 m  
Thickness_H : 0.017 - 0.039 = -0.022 m  
Density_R : 8.984 - 15.083 = -6.098 kg m$^{-3}$  
Density_H : 10.311 - 29.048 = -18.737 kg m$^{-3}$  
SSA_R : 2.249 - 3.520 = -1.271 m$^2$ kg$^{-1}$  
SSA_H : 1.011 - 2.055 = -1.044 m$^2$ kg$^{-1}$  
SWE : 8.041 - 20.014 = -11.973 mm  
Sigma : 0.244 - 0.466 = -0.222 dB  

Posterior - Snowpit

In [ ]:
for i in range(len(variables)-1):

    print(var_names[i] + ' : ' + '{:.3f}'.format((idata_default.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median() - 
      snowpit_stats.sel(site='SM02', property=variables[i], grain_type=gtypes[i])['median'].values).values[0]) + ' ' + units[i])

Thickness_R : 0.006 m  
Thickness_H : -0.040 m  
Density_R : -124.363 kg m$^{-3}$  
Density_H : -41.858 kg m$^{-3}$  
SSA_R : -18.715 m$^2$ kg$^{-1}$  
SSA_H : -1.440 m$^2$ kg$^{-1}$  
SWE : -36.135 mm  

Prior - Snowpit

In [ ]:
for i in range(len(variables)-1):

    print(var_names[i] + ' : ' + '{:.3f}'.format((idata_default.prior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median() - 
      snowpit_stats.sel(site='SM02', property=variables[i], grain_type=gtypes[i])['median'].values).values[0]) + ' ' + units[i])

Thickness_R : 0.150 m  
Thickness_H : -0.078 m  
Density_R : -129.280 kg m$^{-3}$   
Density_H : -41.732 kg m$^{-3}$  
SSA_R : -20.416 m$^2$ kg$^{-1}$  
SSA_H : -2.133 m$^2$ kg$^{-1}$  
SWE : -11.652 mm

Sigma values

In [ ]:
print('{:.3f}'.format((idata_default.prior['Sigma'].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median()).values[0]))
print('{:.3f}'.format((idata_default.posterior['Sigma'].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median()).values[0]))
print('{:.3f}'.format((idata_default.posterior['Sigma'].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().quantile(0.25)).values[0]))

Median prior $\sigma$ = 1.094 dB  
Median posterior $\sigma$ = 1.094 dB  
1st quartile of posterior $\sigma$ = 0.904 dB  

# Posterior vs Prior differences Arctic SVS-2

In [ ]:
for i in range(len(variables)):

    print(var_names[i] + ' : ' + '{:.3f}'.format((idata_final.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median() - 
      idata_final.prior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median()).values[0]) + ' ' + units[i])

Thickness_R : -0.165 m  
Thickness_H : 0.058 m  
Density_R : 33.160 kg m$^{-3}$  
Density_H : 12.615 kg m$^{-3}$  
SSA_R : 4.557 m$^2$ kg$^{-1}$  
SSA_H : 0.551 m$^2$ kg$^{-1}$  
SWE : -24.741 mm  
Sigma : 0.120 dB

In [ ]:
for i in range(len(variables)):

    print(var_names[i] + ' : ' + '{:.3f}'.format(idata_final.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std().values[0]) + ' - '
                                 '{:.3f}'.format(idata_final.prior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std().values[0]) + ' = '
                                                  '{:.3f}'.format((idata_final.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std() - 
      idata_final.prior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std()).values[0]) + ' ' + units[i])

Thickness_R : 0.033 - 0.142 = -0.109 m  
Thickness_H : 0.033 - 0.069 = -0.036 m  
Density_R : 34.155 - 79.049 = -44.894 kg m$^{-3}$  
Density_H : 11.046 - 30.577 = -19.531 kg m$^{-3}$  
SSA_R : 4.403 - 10.308 = -5.905 m$^2$ kg$^{-1}$  
SSA_H : 1.737 - 4.757 = -3.020 m$^2$ kg$^{-1}$  
SWE : 15.516 - 50.590 = -35.074 mm  
Sigma : 0.163 - 0.475 = -0.312 dB

Posterior - Snowpit

In [ ]:
for i in range(len(variables)-1):

    print(var_names[i] + ' : ' + '{:.3f}'.format((idata_final.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median() - 
      snowpit_stats.sel(site='SM02', property=variables[i], grain_type=gtypes[i])['median'].values).values[0]) + ' ' + units[i])

Thickness_R : 0.001 m  
Thickness_H : -0.009 m  
Density_R : -50.328 kg m$^{-3}$  
Density_H : -31.100 kg m$^{-3}$  
SSA_R : -6.979 m$^2$ kg$^{-1}$  
SSA_H : 3.241 m$^2$ kg$^{-1}$  
SWE : -18.495 mm

Prior-Snowpit

In [ ]:
for i in range(len(variables)-1):

    print(var_names[i] + ' : ' + '{:.3f}'.format((idata_final.prior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median() - 
      snowpit_stats.sel(site='SM02', property=variables[i], grain_type=gtypes[i])['median'].values).values[0]) + ' ' + units[i])

Thickness_R : 0.166 m  
Thickness_H : -0.066 m  
Density_R : -83.488 kg m$^{-3}$  
Density_H : -43.715 kg m$^{-3}$  
SSA_R : -11.536 m$^2$ kg$^{-1}$  
SSA_H : 2.689 m$^2$ kg$^{-1}$  
SWE : 6.246 mm

Sigma

In [ ]:
print('{:.3f}'.format((idata_final.prior['Sigma'].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median()).values[0]))
print('{:.3f}'.format((idata_final.posterior['Sigma'].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median()).values[0]))
print('{:.3f}'.format((idata_final.posterior['Sigma'].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().quantile(0.25)).values[0]))

Median prior $\sigma$ = 1.116 dB  
Median posterior $\sigma$ = 1.236 dB  
1st quartile of posterior $\sigma$ = 1.125 dB  

# Posterior vs Prior differences Default SVS-2 free MCMC snow parameterization

In [ ]:
for i in range(len(variables)):

    print(var_names[i] + ' : ' + '{:.3f}'.format((idata_free.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median() - 
      idata_free.prior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median()).values[0]) + ' ' + units[i])

Thickness_R : -0.001 m  
Thickness_H : -0.011 m  
Density_R : 1.902 kg m$^{-3}$  
Density_H : 40.770 kg m$^{-3}$  
SSA_R : 2.306 m$^2$ kg$^{-1}$  
SSA_H : 2.619 m$^2$ kg$^{-1}$  
SWE : 4.860 mm  
Sigma : 0.069 dB

In [ ]:
for i in range(len(variables)):

    print(var_names[i] + ' : ' + '{:.3f}'.format(idata_free.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std().values[0]) + ' - '
                                 '{:.3f}'.format(idata_free.prior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std().values[0]) + ' = '
                                                  '{:.3f}'.format((idata_free.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std() - 
      idata_free.prior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std()).values[0]) + ' ' + units[i])

Thickness_R : 0.041 - 0.077 = -0.036 m  
Thickness_H : 0.019 - 0.039 = -0.020 m  
Density_R : 7.080 - 14.785 = -7.705 kg m$^{-3}$  
Density_H : 11.442 - 29.949 = -18.508 kg m$^{-3}$  
SSA_R : 2.241 - 3.494 = -1.253 m$^2$ kg$^{-1}$  
SSA_H : 1.175 - 1.944 = -0.770 m$^{2}$ kg$^{-1}$  
SWE : 10.826 - 19.508 = -8.682 mm  
Sigma : 0.211 - 0.478 = -0.267 dB

Posterior - snowpit

In [ ]:
for i in range(len(variables)-1):

    print(var_names[i] + ' : ' + '{:.3f}'.format((idata_free.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median() - 
      snowpit_stats.sel(site='SM02', property=variables[i], grain_type=gtypes[i])['median'].values).values[0]) + ' ' + units[i])

Thickness_R : 0.149 m  
Thickness_H : -0.090 m  
Density_R : -127.621 kg m$^{-3}$  
Density_H : 0.296 kg m$^{-3}$  
SSA_R : -18.195 m$^2$ kg$^{-1}$  
SSA_H : 0.238 m$^2$ kg$^{-1}$  
SWE : -8.439 mm

Prior - Snowpit

In [ ]:
for i in range(len(variables)-1):

    print(var_names[i] + ' : ' + '{:.3f}'.format((idata_free.prior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median() - 
      snowpit_stats.sel(site='SM02', property=variables[i], grain_type=gtypes[i])['median'].values).values[0]) + ' ' + units[i])

Thickness_R : 0.150 m  
Thickness_H : -0.078 m  
Density_R : -129.524 kg m$^{-3}$  
Density_H : -40.474 kg m$^{-3}$  
SSA_R : -20.502 m$^2$ kg$^{-1}$  
SSA_H : -2.381 m$^2$ kg$^{-1}$  
SWE : -13.299 mm

Sigma

In [ ]:
print('{:.3f}'.format((idata_free.prior['Sigma'].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median()).values[0]))
print('{:.3f}'.format((idata_free.posterior['Sigma'].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median()).values[0]))
print('{:.3f}'.format((idata_free.posterior['Sigma'].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().quantile(0.25)).values[0]))

Median prior $\sigma$ = 1.121 dB  
Median posterior $\sigma$ = 1.189 dB  
1st quartile of posterior $\sigma$ = 1.044 dB  

# Posterior differences between default and Arctic SVS-2

In [ ]:
for i in range(len(variables)):

    print(var_names[i] + ' : ' + '{:.3f}'.format((idata_default.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median() - 
      idata_final.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median()).values[0]) + ' ' + units[i])

Thickness_R : 0.005 m  
Thickness_H : -0.031 m  
Density_R : -74.035 kg m$^{-3}$  
Density_H : -10.758 kg m$^{-3}$  
SSA_R : -11.736 m$^2$ kg$^{-1}$  
SSA_H : -4.681 m$^2$ kg$^{-1}$  
SWE : -17.640 mm  
Sigma : -0.142 dB

In [ ]:
for i in range(len(variables)):

    print(var_names[i] + ' : ' + '{:.3f}'.format(idata_default.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std().values[0]) + ' - '
                                 '{:.3f}'.format(idata_final.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std().values[0]) + ' = '
                                                  '{:.3f}'.format((idata_default.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std() - 
      idata_final.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std()).values[0]) + ' ' + units[i])

Thickness_R : 0.020 - 0.033 = -0.013 m  
Thickness_H : 0.017 - 0.033 = -0.015 m  
Density_R : 8.984 - 34.155 = -25.170 kg m$^{-3}$  
Density_H : 10.311 - 11.046 = -0.735 kg m$^{-3}$  
SSA_R : 2.249 - 4.403 = -2.154 m$^2$ kg$^{-1}$  
SSA_H : 1.011 - 1.737 = -0.726 m$^2$ kg$^{-1}$  
SWE : 8.041 - 15.516 = -7.475 mm  
Sigma : 0.244 - 0.163 = 0.081 dB

# Posterior differences between constrained and free MCMC

In [ ]:
for i in range(len(variables)):

    print(var_names[i] + ' : ' + '{:.3f}'.format((idata_default.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median() - 
      idata_free.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median()).values[0]) + ' ' + units[i])

Thickness_R : -0.143 m  
Thickness_H : 0.050 m  
Density_R : 3.259 kg m$^{-3}$  
Density_H : -42.154 kg m$^{-3}$  
SSA_R : -0.519 m$^2$ kg$^{-1}$  
SSA_H : -1.678 m$^2$ kg$^{-1}$  
SWE : -27.697 mm  
Sigma : -0.095 dB

In [ ]:
for i in range(len(variables)):

    print(var_names[i] + ' : ' + '{:.3f}'.format(idata_default.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std().values[0]) + ' - '
                                 '{:.3f}'.format(idata_free.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std().values[0]) + ' = '
                                                  '{:.3f}'.format((idata_default.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std() - 
      idata_free.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std()).values[0]) + ' ' + units[i])

Thickness_R : 0.020 - 0.041 = -0.020 m  
Thickness_H : 0.017 - 0.019 = -0.002 m  
Density_R : 8.984 - 7.080 = 1.904 kg m$^{-3}$  
Density_H : 10.311 - 11.442 = -1.131 kg m$^{-3}$  
SSA_R : 2.249 - 2.241 = 0.008 m$^2$ kg$^{-1}$  
SSA_H : 1.011 - 1.175 = -0.163 m$^2$ kg$^{-1}$  
SWE : 8.041 - 10.826 = -2.785 mm  
Sigma : 0.244 - 0.211 = 0.033 dB

# Posterior differences between Arctic SVS-2 and free MCMC

In [ ]:
for i in range(len(variables)):

    print(var_names[i] + ' : ' + '{:.3f}'.format((idata_final.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median() - 
      idata_free.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().median()).values[0]) + ' ' + units[i])

Thickness_R : -0.148 m  
Thickness_H : 0.081 m  
Density_R : 77.294 kg m$^{-3}$  
Density_H : -31.395 kg m$^{-3}$  
SSA_R : 11.216 m$^2$ kg$^{-1}$  
SSA_H : 3.002 m$^2$ kg$^{-1}$  
SWE : -10.056 mm  
Sigma : 0.047 dB

In [ ]:
for i in range(len(variables)):

    print(var_names[i] + ' : ' + '{:.3f}'.format(idata_final.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std().values[0]) + ' - '
                                 '{:.3f}'.format(idata_free.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std().values[0]) + ' = '
                                                  '{:.3f}'.format((idata_final.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std() - 
      idata_free.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().std()).values[0]) + ' ' + units[i])

Thickness_R : 0.033 - 0.041 = -0.008 m  
Thickness_H : 0.033 - 0.019 = 0.013 m  
Density_R : 34.155 - 7.080 = 27.075 kg m$^{-3}$  
Density_H : 11.046 - 11.442 = -0.396 kg m$^{-3}$  
SSA_R : 4.403 - 2.241 = 2.162 m$^2$ kg$^{-1}$  
SSA_H : 1.737 - 1.175 = 0.562 m$^2$ kg$^{-1}$  
SWE : 15.516 - 10.826 = 4.690 mm  
Sigma : 0.163 - 0.211 = -0.048 dB

# This section shows extra tests that were done to support discussions in [Montpetit et al. (Preprint)](https://doi.org)

## Shows MCMC outputs on SM site [(see Figure 2 and Tables 2 and 6 of Montpetit et al., 2024)](https://tc.copernicus.org/articles/18/3857/2024/#&gid=1&pid=1)

### Test when SWE prior is 2x greater than SVS-2 output

In [ ]:
var_names = ['Thickness_R','Thickness_H',
             'Density_R','Density_H',
             'SSA_R','SSA_H',
             'SWE','Sigma']
variables = ['thickness','thickness',
            'density','density',
            'ssa','ssa',
            'swe']
gtypes=['R','H','R','H','R','H','R','H']
xmins=[0,0,100,100,5,5,0,0]
xmaxs=[1,1,450,450,50,50,300,2.5]

site = 'SM02'

idata_site = az.from_netcdf('Data/MCMC_Output_SM02_4obs_2xSWEoffTest_cond.nc')
idata_site.posterior = idata_site.posterior.assign(SWE=(idata_site.posterior['Density_R']*idata_site.posterior['Thickness_R']+
                                                      idata_site.posterior['Density_H']*idata_site.posterior['Thickness_H']))
idata_site.prior = idata_site.prior.assign(SWE=(idata_site.prior['Density_R']*idata_site.prior['Thickness_R']+
                                                      idata_site.prior['Density_H']*idata_site.prior['Thickness_H']))

fig, ax = plt.subplots(8,2,figsize=(10,20))

for i in range(len(var_names)):

    idata_site.prior[var_names[i]].mean(dim='chain').to_dataframe().plot(ax=ax[i,0],kind='density',color='grey',
                                                                         label='Prior',legend=None,alpha=0.7);
    idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().plot(ax=ax[i,0],kind='density',color='k',
                                                                             label='Posterior',legend=None);
    idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().plot(ax=ax[i,1],color='k',legend=None);

    y1=idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).min(dim='chain').values
    y2=idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).max(dim='chain').values
    ax[i,1].fill_between(range(5000),y1=y2,y2=y1,color='k',alpha=0.5)
    
    if i<7:

        x=np.linspace(xmins[i],xmaxs[i],200)
        ax[i,0].plot(x, norm.pdf(x,loc=snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['median'].values,
                           scale=snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['std'].values), 
                   color='cyan', linewidth=2,alpha=0.7)
        
    ax[i,0].set_xlabel(var_names[i]);
    ax[i,0].set_ylabel('');
    ax[i,1].set_ylabel(var_names[i]);
    ax[i,1].set_xlabel('');
    ax[i,0].yaxis.set_major_formatter(FormatStrFormatter('%0.2f'));

    ax[0,0].set_xlim(0,1)
    ax[1,0].set_xlim(0,1)
    ax[2,0].set_xlim(100,450)
    ax[3,0].set_xlim(100,450)
    ax[4,0].set_xlim(5,50)
    ax[5,0].set_xlim(5,50)
    ax[6,0].set_xlim(0,300)
    ax[7,0].set_xlim(0,2.5)
    
    ax[3,0].set_ylabel('PDF')
    ax[7,1].set_xlabel('Iteration')
    ax[0,0].legend(['Prior','Posterior','Snowpits'],loc=1);
    fig.savefig('Figures/TracePlot_'+site+'_2xSWEoffTest_cond.png')

<center><img src="Figures/TracePlot_SM02_2xSWEoffTest_cond.png" Width="500px"></center>

<center>Figure: Traceplot of the MCMC optimization when SWE prior is 2x greater than SVS-2 output.</center>

Here we see that after 5000 iterations, the posterior SWE is closer to the sruveyed SWE but shows a larger error than previous tests. This is most likely due to the lower sensitivity to the rounded grain layer. When the prior estimate is so great, the MCMC method would most likely need more iterations to properly estimate SWE.

### Test when SWE prior is 0.5x smaller than SVS-2 output

In [ ]:
var_names = ['Thickness_R','Thickness_H',
             'Density_R','Density_H',
             'SSA_R','SSA_H',
             'SWE','Sigma']
variables = ['thickness','thickness',
            'density','density',
            'ssa','ssa',
            'swe']
gtypes=['R','H','R','H','R','H','R','H']
xmins=[0,0,100,100,5,5,0,0]
xmaxs=[1,1,450,450,50,50,300,2.5]

site = 'SM02'

idata_site = az.from_netcdf('Data/MCMC_Output_SM02_4obs_0.5xSWEoffGoodSSATest_cond.nc')
idata_site.posterior = idata_site.posterior.assign(SWE=(idata_site.posterior['Density_R']*idata_site.posterior['Thickness_R']+
                                                      idata_site.posterior['Density_H']*idata_site.posterior['Thickness_H']))
idata_site.prior = idata_site.prior.assign(SWE=(idata_site.prior['Density_R']*idata_site.prior['Thickness_R']+
                                                      idata_site.prior['Density_H']*idata_site.prior['Thickness_H']))

fig, ax = plt.subplots(8,2,figsize=(10,20))

for i in range(len(var_names)):

    idata_site.prior[var_names[i]].mean(dim='chain').to_dataframe().plot(ax=ax[i,0],kind='density',color='grey',
                                                                         label='Prior',legend=None,alpha=0.7);
    idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().plot(ax=ax[i,0],kind='density',color='k',
                                                                             label='Posterior',legend=None);
    idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().plot(ax=ax[i,1],color='k',legend=None);

    y1=idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).min(dim='chain').values
    y2=idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).max(dim='chain').values
    ax[i,1].fill_between(range(5000),y1=y2,y2=y1,color='k',alpha=0.5)
    
    if i<7:

        x=np.linspace(xmins[i],xmaxs[i],200)
        ax[i,0].plot(x, norm.pdf(x,loc=snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['median'].values,
                           scale=snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['std'].values), 
                   color='cyan', linewidth=2,alpha=0.7)
        
    ax[i,0].set_xlabel(var_names[i]);
    ax[i,0].set_ylabel('');
    ax[i,1].set_ylabel(var_names[i]);
    ax[i,1].set_xlabel('');
    ax[i,0].yaxis.set_major_formatter(FormatStrFormatter('%0.2f'));

    ax[0,0].set_xlim(0,1)
    ax[1,0].set_xlim(0,1)
    ax[2,0].set_xlim(100,450)
    ax[3,0].set_xlim(100,450)
    ax[4,0].set_xlim(5,50)
    ax[5,0].set_xlim(5,50)
    ax[6,0].set_xlim(0,300)
    ax[7,0].set_xlim(0,2.5)
    
    ax[3,0].set_ylabel('PDF')
    ax[7,1].set_xlabel('Iteration')
    ax[0,0].legend(['Prior','Posterior','Snowpits'],loc=1);
    fig.savefig('Figures/TracePlot_'+site+'_0.5xSWEoffGoodSSATest_cond.png')

<center><img src="Figures/TracePlot_SM02_0.5xSWEoffGoodSSATest_cond.png" Width="500px"></center>

<center>Figure: Traceplot of the MCMC optimization when SWE prior is 0.5x smaller than SVS-2 output.</center>

Here we see that the posterior SWE is closer to the surveyed SWE and the depth hoar properties are better estimated. More iterations would most likely be needed to properly optimize the rounded grain layer.  

These two figures show the importance of having proper prior estimates to reduce the number of iterations needed to improve computation effeciency.

### Comparing different scenarios

In [ ]:
var_names = ['Thickness_R','Thickness_H',
             'Density_R','Density_H',
             'SSA_R','SSA_H',
             'SWE','Sigma']
variables = ['thickness','thickness',
            'density','density',
            'ssa','ssa',
            'swe']
gtypes=['R','H','R','H','R','H','R','H']
xmins=[0,0,100,100,5,5,0,0]
xmaxs=[1,1,450,450,50,50,300,2.5]

site = 'SM02'

filepaths = ['Data/MCMC_Output_'+site+'_1obs_Default.nc',
             'Data/MCMC_Outputs/MCMC_Output_'+site+'_4obs_Arctic_WSRMSE_conditions.nc',
             glob.glob('Data/MCMC_Outputs/MCMC_Output_'+site+'_*_4obs_Arctic.nc')[0]]



fig, ax = plt.subplots(8,3,figsize=(10,20))

for j in range(len(filepaths)):

    idata_site = az.from_netcdf(filepaths[j])
    idata_site.posterior = idata_site.posterior.assign(SWE=(idata_site.posterior['Density_R']*idata_site.posterior['Thickness_R']+
                                                          idata_site.posterior['Density_H']*idata_site.posterior['Thickness_H']))
    idata_site.prior = idata_site.prior.assign(SWE=(idata_site.prior['Density_R']*idata_site.prior['Thickness_R']+
                                                          idata_site.prior['Density_H']*idata_site.prior['Thickness_H']))

    for i in range(len(var_names)):
    
        idata_site.prior[var_names[i]].mean(dim='chain').to_dataframe().plot(ax=ax[i,j],kind='density',color='grey',
                                                                             label='Prior',legend=None,alpha=0.7);
        idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().plot(ax=ax[i,j],kind='density',color='k',
                                                                                 label='Posterior',legend=None);
        
        if i<7:
    
            x=np.linspace(xmins[i],xmaxs[i],200)
            ax[i,j].plot(x, norm.pdf(x,loc=snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['median'].values,
                               scale=snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['std'].values), 
                       color='cyan', linewidth=2,alpha=0.7)

        ax[i,j].yaxis.set_major_formatter(FormatStrFormatter('%0.2f'));
        ax[i,j].set_ylabel('');

    ax[0,j].set_xlim(0,0.6)
    ax[1,j].set_xlim(0,0.6)
    ax[2,j].set_xlim(100,400)
    ax[3,j].set_xlim(100,400)
    ax[4,j].set_xlim(5,50)
    ax[5,j].set_xlim(5,50)
    ax[6,j].set_xlim(0,300)
    ax[7,j].set_xlim(0,2.6)
    ax[0,j].set_ylim(0,21)
    ax[1,j].set_ylim(0,25)
    ax[2,j].set_ylim(0,.065)
    ax[3,j].set_ylim(0,.04)
    ax[4,j].set_ylim(0,.25)
    ax[5,j].set_ylim(0,.4)
    ax[6,j].set_ylim(0,0.05)
    ax[7,j].set_ylim(0,2.5)

ax[0,1].set_xlabel('H$_{snow}$ [R] (m)')
ax[1,1].set_xlabel('H$_{snow}$ [H] (m)')
ax[2,1].set_xlabel('$\\rho_{snow}$ [R] (kg$\\cdot$m$^{-3}$)')
ax[3,1].set_xlabel('$\\rho_{snow}$ [H] (kg$\\cdot$m$^{-3}$)')
ax[4,1].set_xlabel('SSA [R] (m$^{2}\\cdot$kg$^{-3}$)')
ax[5,1].set_xlabel('SSA [H] (m$^{2}\\cdot$kg$^{-3}$)')
ax[6,1].set_xlabel('SWE (mm)')
ax[7,1].set_xlabel('$\\sigma$ (dB)')
ax[3,0].set_ylabel('PDF')
# ax[7,1].set_xlabel('Iteration')
ax[0,2].legend(['Prior','Posterior','Snowpits'],loc=1);
ax[0,0].text(0.01,17,'a)')
ax[0,1].text(0.01,17,'b)')
ax[0,2].text(0.01,17,'c)')
fig.savefig('Figures/f10.png')

<center><img src="Figures/f10.png" Width="500px"></center>

<center>Figure: Comparing priors and posteriors for a) the default SVS-2 priors with inter-layer constraints, b) the Arctic SVS-2 priors, with inter-layer constraints and four observations, and c) the Arctic SVS-2 priors, four observations, without the inter-layer constraints.</center>

In [ ]:
var_names = ['Thickness_R','Thickness_H',
             'Density_R','Density_H',
             'SSA_R','SSA_H',
             'SWE','Sigma']
variables = ['thickness','thickness',
            'density','density',
            'ssa','ssa',
            'swe']
gtypes=['R','H','R','H','R','H','R','H']
xmins=[0,0,100,100,5,5,0,0]
xmaxs=[1,1,450,450,50,50,300,2.5]

site = 'SM02'

filepaths = ['Data/MCMC_Output_'+site+'_1obs_Default.nc',
             'Data/MCMC_Outputs/MCMC_Output_'+site+'_4obs_Arctic_WSRMSE_conditions.nc',
             glob.glob('Data/MCMC_Outputs/MCMC_Output_'+site+'_*_4obs_Arctic.nc')[0]]



fig, ax = plt.subplots(8,3,figsize=(10,20))

for j in range(len(filepaths)):

    idata_site = az.from_netcdf(filepaths[j])
    idata_site.posterior = idata_site.posterior.assign(SWE=(idata_site.posterior['Density_R']*idata_site.posterior['Thickness_R']+
                                                          idata_site.posterior['Density_H']*idata_site.posterior['Thickness_H']))
    idata_site.prior = idata_site.prior.assign(SWE=(idata_site.prior['Density_R']*idata_site.prior['Thickness_R']+
                                                          idata_site.prior['Density_H']*idata_site.prior['Thickness_H']))

    for i in range(len(var_names)):

        idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).median(dim='chain').to_dataframe().plot(ax=ax[i,j],color='k',legend=None);
    
        y1=idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).min(dim='chain').values
        y2=idata_site.posterior[var_names[i]].isel(draw=slice(0,5000)).max(dim='chain').values
        ax[i,j].fill_between(range(5000),y1=y2,y2=y1,color='k',alpha=0.5)
        
        if i<7:

            ax[i,j].axhline(snowpit_stats.sel(site=site, property=variables[i], grain_type=gtypes[i])['median'].values,
                            color='cyan', linewidth=2,alpha=0.7)

        ax[i,j].yaxis.set_major_formatter(FormatStrFormatter('%0.2f'));
        ax[i,j].set_ylabel('');
        ax[i,j].set_xlabel('');
        ax[i,j].set_xlim(0,5001)

    ax[0,j].set_ylim(0,0.75)
    ax[1,j].set_ylim(0,0.6)
    ax[2,j].set_ylim(100,450)
    ax[3,j].set_ylim(100,450)
    ax[4,j].set_ylim(5,50)
    ax[5,j].set_ylim(5,50)
    ax[6,j].set_ylim(0,300)
    ax[7,j].set_ylim(0,2.)

ax[0,0].set_ylabel('H$_{snow}$ [R]\n(m)')
ax[1,0].set_ylabel('H$_{snow}$ [H]\n(m)')
ax[2,0].set_ylabel('$\\rho_{snow}$ [R]\n(kg$\\cdot$m$^{-3}$)')
ax[3,0].set_ylabel('$\\rho_{snow}$ [H]\n(kg$\\cdot$m$^{-3}$)')
ax[4,0].set_ylabel('SSA [R]\n(m$^{2}\\cdot$kg$^{-3}$)')
ax[5,0].set_ylabel('SSA [H]\n(m$^{2}\\cdot$kg$^{-3}$)')
ax[6,0].set_ylabel('SWE\n(mm)')
ax[7,0].set_ylabel('$\\sigma$\n(dB)')
# ax[3,0].set_xlabel('PDF')
ax[7,1].set_xlabel('Iterations')
ax[0,0].legend(['Median','Min/Max','Snowpit\nMedian'],loc=1);
ax[0,0].text(10,.6,'a)')
ax[0,1].text(10,.6,'b)')
ax[0,2].text(10,.6,'c)')
fig.savefig('Figures/f11.png')

<center><img src="Figures/f11.png" Width="500px"></center>

<center>Figure: Comparing the sample evolution for a) the default SVS-2 priors with inter-layer constraints, b) the Arctic SVS-2 priors, with inter-layer constraints and four observations, and c) the Arctic SVS-2 priors, four observations, without the inter-layer constraints.</center>